# Tentative solution


This set of iPython notebook aims to resolve the M8r [Issue #108](https://github.com/ahay/src/issues/108): software verification for __sfawefd3d__ and __sfawefd2d__ programs.


### Problem defined
The __sfawefd3d__ and __sfawefd2d__ implement the second-order acoustic scalar wave equations in 3D and 2D media, respectively. The wave equations are suitable for describing the pressure, scalar displacement potential, and scalar velocity potential.

For simple models, the aforementioned wave equations have analytic solutions. Point acooustic source in an unbounded homogeneous medium and point source in fluid halfspace are two examples.

Two issues have been found that the numerical solutions generated by __sfawefd3d__ do not match those analytic Green's functions. See more details from the [testing examples](https://github.com/khokhlov/sfawefd3d_test):
+ The numerical solutions in the homogeneous unbounded case differ from the analytic solutions by a scaling factor of $c^2 /\,h^3$, where $c$ is the speed of sound in ideal fluids and $h$ is the spatial grid sampling (assuming homogeneous sampling along $x-$, $y-$, and $z-$axis).
+ The numerical solutions in fluid halfspace case differ from the analytic solutions quite mysteriously (not simple scaling factor).

### Goal
Explain why there is mismatching between the solutions from our modeling codes and the analytic solutions. Figure out the fishy parts in codes and patch them.

### Explanation for the mismatching in homogeneous-media case
One issue is quite clear to us that we do not correctly scale the source function with corresponding speed of sounds ($c^2$) in the __sfawefd3d__ codes. Therefore, one component of the scaling factor ($c^2$) in the first issue is explainable and I already did a patch to fix it. We are then left with the $1\,/\, h^3$ factor.

Let us consider the second-order acoustic wave equation for displacement potential (same argument holds for pressure and velocity potential):

\begin{equation}
 c^{-2}\frac{\partial^2 \Psi}{\partial t^2} - \nabla^2 \Psi = s,
\end{equation}

where $s$ is the source functions, and $\Psi$ is the scalar displacement potential:

\begin{equation}
 \mathbf{U} = \nabla \Psi,
\end{equation}

where $\mathbf{U}$ is the particle displacement vector. The 3D Green's function in an unbounded homogeneous media is given by

\begin{equation}
 G(\mathbf{x}_r, \mathbf{x}_s) = \frac{S(t - |\mathbf{x}_s - \mathbf{x}_r| / c)}{4\pi|\mathbf{x}_s - \mathbf{x}_r|}.
\end{equation}

Notice that I use the capitalized letter $S$ for the source function rather than the small $s$. This is for the reasons that will be clear soon.

Let us do some dimensional analysis. The displacement vector has the units of m, and the scalar displacement potential $\Psi$ has the units of m$^2$. The Green's function is just the impulse response and therefore has the same units with scalar displacement potential $\Psi$, which is m$^2$. Therefore the captilized $S$ must have the units of m$^3$. Now, we look at the small $s$ in the first wave equation, it should be dimensionless because the left hand side of the wave equation is dimensionless. Therefore, we have different source functions for the wave equation and the Green's function!!!

So the caveat here is that the wave equation we used is in strong form or called differential form. The classical mechanics textbooks often derive these equations from weak forms (integral forms) of the conservation equations and constitutive laws. In the weak form, every quantity is a sought of net quantity, for example, the mass is defined in terms of the mass densities:

\begin{equation}
 M = \int \rho \, dV,
\end{equation}

wheras going from weak form to strong form, we take away the integrals and directly operate with the densities of quantities. So, this is the story of why we have different characters in wave equation and the Green's function. The small $s$ we used in the wave equation is just the source density of the net source $S$ we used in the Green's function:

\begin{equation}
 S = \int s \, dV.
\end{equation}

Therefore, if use denote a discrete time signal (e.g. Ricker wavelet in the testing example) as the source function for the Green's function evaluation, we have already assign some units to that signal, which is m$^3$ in our case. If we want to use that signal in the __sfawefd3d__ program to perform numerical modeling, we need to pre-process the source function to get the source density function (differentiate $S$ to get $s$). 

Basically, for the point source case, the analytic world thinks it's a perfect point source, and the numerical worlds thinks the point source has finite size (one grid point in the finite-difference simulations), the size of the point source is just the volume of that grid point:
    
\begin{equation}
  V = h_x \times h_y \times h_z = h^3.
\end{equation}

So going from $S$ to $s$, we just need to divide $S$ by the volume of the point source because the source density function is spatially invariant:

\begin{equation}
 s = S \, / \, h^3.
\end{equation}

Once we get the source density function and perform the numerical simulations, we will get a numerical solution that is consistent with the Green's function, otherwise, the Green's function is $1\,/\,h^3$ of the numerical solution.

I will illustrate using the same example as Nikolay Khokhlov's [testing examples](https://github.com/khokhlov/sfawefd3d_test)

Nikolay did everything perfect, except that source function passed to the modeling program should be the source density function, i.e., __impulse_density.rsf__ rather than __impulse.rsf__

In [1]:
h = 10
volume = h**3
def madagascar(free='n'):
    cmd = 'sfmath > vel.rsf output=%s n1=%s n2=%s n3=%s d1=%s d2=%s d3=%s o1=%s o2=%s o3=%s' % (c, nz, nx, ny, h, h, h, oz, ox, oy)
    print cmd
    os.system(cmd)
    os.system('sfspike n1=3 nsp=3 k1=1,2,3 mag=%s,%s,%s o1=0 o2=0 o3=0 > sou.rsf' % (s[0], s[1], s[2]))
    os.system('sfspike n1=3 nsp=3 k1=1,2,3 mag=%s,%s,%s o1=0 o2=0 o3=0 > rec.rsf' % (r[0], r[1], r[2]))
    os.system('echo in=impulse.txt n1=1 n2=%s data_format=ascii_float | sfdd form=native | sfput n1=1 n2=%s d2=%s o1=0 label1="Time" unit1="s" > impulse.rsf' % (N, N, dt))
    os.system('sfmath <impluse.rsf output="input/%g" > impluse_density.rsf' % volume)
    cmd = 'sfawefd3d < impulse_density.rsf vel=vel.rsf sou=sou.rsf rec=rec.rsf > dat.rsf verb=y free=%s expl=n snap=n dabc=y jdata=1 sinc=y' % free
    print cmd
    os.system(cmd)
    os.system('sfdisfil > dat.asc col=1 format="%e " number=n < dat.rsf')

With this set-up, the Green's function and the numerical solution matches each other (the $c^2$ scaling has been fixed in a newer version).

### Explanation for the mismatching in fluid halfspace case

The question for mismatching in the presence of free surface condition seems nontrival to answer. But Nikolay was going the right direction. There is one grid shift of the free surface condition in the modeling codes compared to the analytic solution case. The Green's function is obtained from the fact that the pressure is already zero at $z = 0$, which is the first physical grid point in the modeling codes. However, what the modeling code does was setting any pressure above the first grid to zero and leaving the first grid point unchanged.

So, we can fool the program and shift one grid down to make the numerical solution matches with the analytic solution. With all the parameter set-up in Nikolay's example, we only need to change to the following:

In [8]:
oz = h

and re-run Nikolay's last example. We will get quite good matching of the two solutions.

### Summary
+ The mismatching for the point source homogeneous case comes from the fact the the source functions should be different for numerical computations and analytic Green's function calculations.
+ The mismatching for the point source fluid halfspace case comes from the fact that __sfawefd3d__ program does not set the pressure to zero at the very first physical grid point.
+ The M8r [Issue #108](https://github.com/ahay/src/issues/108) has been resolved by the recent commits 